In [22]:
import pandas as pd
import psycopg2 as psy
import os
from dotenv import load_dotenv

try:

    load_dotenv() # - carregando as variáveis de ambiente para que possam ser utilizadas
    
    conexao = psy.connect(
        dbname = os.getenv("DB_NAME"),
        user = os.getenv("USER"),
        password = os.getenv("PASSWORD"),
        host = os.getenv("HOST"),
        port = os.getenv("PORT")
    ) # - estabelecendo a conexão com o banco de dados

    cursor = conexao.cursor() 

    #execução do comando para trazer os dados da tabela original
    cursor.execute(
        '''SELECT * FROM usuario;'''
    )

    #criando um dataframe apartir dos dados capturados
    df = pd.DataFrame(cursor.fetchall(), columns=[item[0] for item in cursor.description])


    cursor.close()
    conexao.close()

except Exception as e:
    print(f"Erro ao conectar com o banco: "+str(e)) # - tratando exceção, (caso ocorra)

In [23]:
#Pseudonimizando as colunas da tabela
df_user_pseudo = df.copy() # - criando uma cópia do dataframe para pseudonimizar de forma eficiente

#Excluindo campos do dataframe que não serão necessários
df_user_pseudo = df_user_pseudo.drop(columns="tipo_acesso")


#Pseudonimização da coluna de id 
df_user_pseudo['id'] = df_user_pseudo['id'].apply(lambda id: f'{"*" * (len(str(id))-1)}{str(id)[-10:]}')
#Pseudonimização da coluna de fk_fabrica
df_user_pseudo['fk_fabrica'] = ['FK' for fk in df_user_pseudo['fk_fabrica']]
#Pseudonimização da coluna de id_gerente
df_user_pseudo['id_gerente'] = ['ID' for id in df_user_pseudo['id_gerente']]
#Pseudonimização da coluna de email
df_user_pseudo['email'] = df_user_pseudo['email'].apply(lambda email: email[:3] + f"{'*' * (email.find('@') -3)}"+email[email.find('@'):])
#Pseudonimização da coluna de senha - criptografia
#Pseudonimização da coluna de nome
df_user_pseudo['nome'] = ['User' + str(cont).zfill(3) for cont in range(len(df_user_pseudo))]
#Pseudonimização da coluna de genero - Sem necessidade
#Pseudonimização da coluna de desc_tipoacesso
df_user_pseudo['desc_tipoacesso'] = df_user_pseudo['desc_tipoacesso'].apply(lambda desc: desc[:1]+f"{'*' * (len(desc[:]) -1)}")
#Pseudonimização da coluna de status
df_user_pseudo['status'] = df_user_pseudo['status'].apply(lambda status: str(status)[:1]+f'{"*" * (len(str(status))-1)}')
#Pseudonimização da coluna de data_criacao
df_user_pseudo['data_criacao'] = df_user_pseudo['data_criacao'].apply(lambda data: f'{str(data)[:4]}/**/** - {"*" * (len(str(data)) -4)}')
#Pseudonimização da coluna de data_nascimento
df_user_pseudo['data_nascimento'] = df_user_pseudo['data_nascimento'].apply(lambda data: f'{str(data)[:4]}/**/**')
#Pseudonimização da coluna de cargo
df_user_pseudo['cargo'] = df_user_pseudo['cargo'].apply(lambda cargo: cargo[:2]+f'{"*" * (len(cargo)-2)}')



In [24]:
#Renomeando as colunas do dataframe
df_user_pseudo.rename(columns={
    "id":"id",
    "fk_fabrica":"fk_fabrica_pseudo",
    "id_gerente":"id_gerente_pseudo",
    "email": "email_pseudo",
    "senha": "senha_criptografada",
    "nome":"nome_pseudo",
    "genero":"genero",
    "desc_tipoacesso":"desc_tipoacesso_pseudo",
    "status":"status_pseudo",
    "data_criacao":"data_criacao_pseudo",
    "data_nascimento":"data_nascimento_pseudo",
    "cargo" : "cargo_pseudo"
    
}, inplace=True)

In [25]:
#mostrando as operações realizadas com os dados
display(df_user_pseudo)

,id,fk_fabrica_pseudo,id_gerente_pseudo,email_pseudo,senha_criptografada,nome_pseudo,genero,desc_tipoacesso_pseudo,status_pseudo,data_criacao_pseudo,data_nascimento_pseudo,cargo_pseudo
0,***********************************5e971d6ddf,FK,ID,ana******@seara.com.br,$2a$15$6f0MKf1mUEVfSDQeN3OVc.2MYTjZxklW/cwvNnU...,User000,feminino,l*********************************************...,T***,2025/**/** - **********************,1985/**/**,Ge*****
1,***********************************590344b44d,FK,ID,eme**********@jbs.com.br,$2a$15$pQkDPRa454W4Ls4si6OVseEQp/q8T04rxhiaA.j...,User001,masculino,l****************************,T***,2025/**/** - **********************,1990/**/**,Su************
2,***********************************b75fbac9e9,FK,ID,ren***********@jbs.com.br,$2a$15$2nDRCcy8zthAjozlq2.P4eSOEBNaEy8Mj5.uF27...,User002,masculino,l****************************,T***,2025/**/** - **********************,1990/**/**,Su************


In [26]:
#Tranferindo todas as informações do dataframe pseudonimizado para o banco de dados 


try:
    
    conexao = psy.connect(
        dbname = os.getenv("DB_PSEUDO"),
        user = os.getenv("USER"),
        password = os.getenv("PASSWORD"),
        host = os.getenv("HOST"),
        port = os.getenv("PORT")
    ) # - estabelecendo a conexão com o banco de dados

    cursor = conexao.cursor()

    #Execução do comando para trazer os dados da tabela original
    query = """
    DELETE FROM usuario_pseudo;
    """ # - comando que vai ser aplicado

    valores = [tuple(x) for x in df_user_pseudo.to_numpy()] # - transformando cada linha do dataframe em uma tupla

    for c in valores:
        cursor.execute(query, c) # - executando o comando no banco de dados

    conexao.commit() # - salvando as alterações


    cursor.close() # - fechando conexão 
    conexao.close() # - fechando conexão 

except Exception as e:
    print(f"Erro ao conectar com o banco: "+str(e)) # - tratando exceção, (caso ocorra)

In [27]:
#Tranferindo todas as informações do dataframe pseudonimizado para o banco de dados 


try:
    
    conexao = psy.connect(
        dbname = os.getenv("DB_PSEUDO"),
        user = os.getenv("USER"),
        password = os.getenv("PASSWORD"),
        host = os.getenv("HOST"),
        port = os.getenv("PORT")
    ) # - estabelecendo a conexão com o banco de dados

    cursor = conexao.cursor()

    #Execução do comando para trazer os dados da tabela original
    query = """
    INSERT INTO usuario_pseudo (id, fk_fabrica_pseudo, id_gerente_pseudo, email_pseudo, senha_criptografada, nome_pseudo, genero, desc_tipoacesso_pseudo, status_pseudo, data_criacao_pseudo, data_nascimento_pseudo, cargo_pseudo)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) """ # - comando que vai ser aplicado

    valores = [tuple(x) for x in df_user_pseudo.to_numpy()] # - transformando cada linha do dataframe em uma tupla

    for c in valores:
        cursor.execute(query, c) # - executando o comando no banco de dados

    conexao.commit() # - salvando as alterações


    cursor.close() # - fechando conexão 
    conexao.close() # - fechando conexão 

except Exception as e:
    print(f"Erro ao conectar com o banco: "+str(e)) # - tratando exceção, (caso ocorra)

In [28]:
#Tranferindo as informações originais e pseudonimizadas para os arquivos csv
df.to_csv('CSV\original_user.csv')
df_user_pseudo.to_csv('CSV\pseudo_user.csv')